This notebook loads in a trained model and later performs correlation tests on reconstructed data and original data. Also tracks the loss error value.

In [16]:
import pandas as pd
import numpy as np
import random
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from scipy.stats import pearsonr, spearmanr

In [2]:
def generate_random_samples(batch, read = "/Users/maks/Documents/MSc_project/data/features", sequence_length = 60):
    """
    A function that creates a list of independent arrays that are to serve as samples for processing later.
    read is the directory where Excel files containing variables are stored;
    batch is the number of samples that will be generated;
    sequence length is the length of each sample.
    Returns a list of numpy arrays.
    """

    def load_excel(file_path):
        """
        Extracts sheets from Excel files given a file directory.
        """
        excel_file = pd.ExcelFile(file_path)
        
        sheets = {sheet_name: pd.read_excel(excel_file, sheet_name = sheet_name, engine = "openpyxl") for sheet_name in excel_file.sheet_names}

        return sheets
    
    def random_file(read):
        """
        Chooses a random file from a provided directory.
        """
        files = [f for f in os.listdir(read) if f.endswith(".xlsx") and not f.startswith(".")]
        
        if not files:
            raise FileNotFoundError("No Excel files found in the directory")
        
        choice = random.choice(files)
        #print(f"The file used is: {choice}")
        
        return os.path.join(read, choice)
    
    def normalise_columns(df, columns):
        """
        Normalises specified columns between -1 and 1.
        """
        df = df.copy()  # Make a copy to avoid modifying the original DataFrame
    
        for column in columns:
            max_val = df[column].max()
            min_val = df[column].min()
            
            df.loc[:, column] = 2 * (df[column] - min_val) / (max_val - min_val) - 1

        return df

    def sampler(sheet_data, sequence_length):
        """
        Randomly creates a sample from an Excel sheet.
        sheet_data are sheets representing time series segments from where data is going to be sampled
        sequence_length is the number of points each sample will contain.
        """

        #Normalises velocities and angular velocities using normalise_columns
        columns = ["velocity_x", "velocity_y", "velocity_z", "angular_velocity_yz", "angular_velocity_xz", "angular_velocity_xy"]
        sheet_data = normalise_columns(sheet_data, columns)

        num_points = len(sheet_data)

        if num_points < sequence_length + 2:
            return None
        
        start_index = random.randint(2, num_points - sequence_length)
        end_index = start_index + sequence_length
        sample = sheet_data.iloc[start_index : end_index].to_numpy()

        return sample #Transposes the samle to switch rows and columns
    
    def weighted_random_choice(sheets):
        """
        Assigns weights to sheets based on the length of each sheet.
        """
        total_points = sum(len(data) for data in sheets.values())
        weights = [len(data) / total_points for data in sheets.values()]

        return random.choices(list(sheets.keys()), weights = weights, k = 1)[0]
    
    #Initializes list to strore sample arrays
    samples = []

    while len(samples) < batch:
        file_path = random_file(read)
        sheets = load_excel(file_path)

        while len(samples) < batch:
            sheet_name = weighted_random_choice(sheets)
            sheet_data = sheets[sheet_name][["velocity_x", "velocity_y", "velocity_z", "tortuosity3d", "angular_velocity_yz", "angular_velocity_xz", "angular_velocity_xy"]]
            sample = sampler(sheet_data, sequence_length)

            if sample is not None:
                samples.append(sample)

            #Stops once batch size is reached
            if len(samples) >= batch:
                break

    return samples

In [3]:
#Class to contain our sampled time series data
class TimeSeriesData(Dataset):
    def __init__(self, no_samples):
        self.data = generate_random_samples(no_samples)
        self.data = np.array(self.data)

        self.data = torch.tensor(self.data, dtype = torch.float32)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, i):
        return self.data[i]

In [4]:
class BranchingAE(nn.Module):
    def __init__(self):
        super(BranchingAE, self).__init__()

        #Branch for velocities (first 3 features)
        self.velocity_branch = nn.Sequential(
            nn.Conv1d(in_channels = 3, out_channels = 32, kernel_size = 3, stride = 1, padding = 1),
            nn.ELU(),
            nn.MaxPool1d(kernel_size = 2, stride = 2),

            nn.Conv1d(in_channels = 32, out_channels = 64, kernel_size = 3, stride = 1, padding = 1),
            nn.ELU(),
            nn.MaxPool1d(kernel_size = 2, stride = 2)
        )

        #Branch for tortuosity (4th feature)
        self.tortuosity_branch = nn.Sequential(
            nn.Conv1d(in_channels = 1, out_channels = 32, kernel_size = 3, stride = 1, padding = 1),
            nn.ELU(),
            nn.MaxPool1d(kernel_size = 2, stride = 2),

            nn.Conv1d(in_channels = 32, out_channels = 64, kernel_size = 3, stride = 1, padding = 1),
            nn.ELU(),
            nn.MaxPool1d(kernel_size = 2, stride = 2)
        )

        #Branch for angular velocities (last 3 features (out of 7))
        self.ang_velocity_branch = nn.Sequential(
            nn.Conv1d(in_channels = 3, out_channels = 32, kernel_size = 3, stride = 1, padding = 1),
            nn.ELU(),
            nn.MaxPool1d(kernel_size = 2, stride = 2),

            nn.Conv1d(in_channels = 32, out_channels = 64, kernel_size = 3, stride = 1, padding = 1),
            nn.ELU(),
            nn.MaxPool1d(kernel_size = 2, stride = 2)
        )

        #Shared decoder for merging the features
        self.decoder = nn.Sequential(
            nn.ConvTranspose1d(in_channels = 192, out_channels = 128, kernel_size = 2, stride = 2),
            nn.ELU(),

            nn.ConvTranspose1d(in_channels = 128, out_channels = 64, kernel_size = 2, stride = 2),
            nn.ELU(),

            nn.ConvTranspose1d(in_channels = 64, out_channels = 32, kernel_size = 3, stride = 1, padding = 1),
            nn.ELU()
        )

        #Output layers for different variables
        self.output_velocity = nn.Sequential(
            nn.Conv1d(in_channels = 32, out_channels = 3, kernel_size = 3, stride = 1, padding = 1),
            nn.Tanh()
        )

        self.output_tortuosity = nn.Sequential(
            nn.Conv1d(in_channels = 32, out_channels = 1, kernel_size = 3, stride = 1, padding = 1)
        )

        self.output_ang_velocity = nn.Sequential(
            nn.Conv1d(in_channels = 32, out_channels = 3, kernel_size = 3, stride = 1, padding = 1),
            nn.Tanh()
        )

    def forward(self, x):

        #print("Input shape:", x.shape)

        velocity = x[:, :, :3]
        tortuosity = x[:, :, 3:4]
        ang_velocity = x[:, :, 4:]

        #print("Velocity shape:", velocity.shape)
        #print("Tortuosity shape:", tortuosity.shape)
        #print("Angular velocity shape:", ang_velocity.shape)

        velocity = velocity.permute(0, 2, 1)
        tortuosity = tortuosity.permute(0, 2, 1)
        ang_velocity = ang_velocity.permute(0, 2, 1)

        #print("Permuted elocity shape:", velocity.shape)
        #print("permuted tortuosity shape:", tortuosity.shape)
        #print("Permuted angular velocity shape:", ang_velocity.shape)

        #Permutes to desired shape
        velocity_encoded = self.velocity_branch(velocity)
        tortuosity_encoded = self.tortuosity_branch(tortuosity)
        ang_velocity_encoded = self.ang_velocity_branch(ang_velocity)

        #print("Velocity encoded shape:", velocity_encoded.shape)
        #print("Tortuosity encoded shape:", tortuosity_encoded.shape)
        #print("Angular velocity encoded shape:", ang_velocity_encoded.shape)

        #Concatenates encoded features
        combined_encoded = torch.cat((velocity_encoded, tortuosity_encoded, ang_velocity_encoded), dim = 1)

        #print("Combined encoded shape:", combined_encoded.shape)

        #Decodes combined features
        decoded = self.decoder(combined_encoded)

        #print("Decoded shape:", decoded.shape)

        #Generates outputs for each set of variables
        velocity_output = self.output_velocity(decoded)
        tortuosity_output = self.output_tortuosity(decoded)
        ang_velocity_output = self.output_ang_velocity(decoded)

        #print("Velocity output shape:", velocity_output.shape)
        #print("Tortuosity output shape:", tortuosity_output.shape)
        #print("Angular velocity output shape:", ang_velocity_output.shape)  

        #Concatenates all outputs
        combined_output = torch.cat((velocity_output, tortuosity_output, ang_velocity_output), dim = 1)
        
        #print("Combined output shape:", combined_output.shape)

        #Permutes back
        output = combined_output.permute(0, 2, 1)
        
        #print("Final output shape:", output.shape)

        return output
    
    def encode(self, x):

        velocity = x[:, :, :3]
        tortuosity = x[:, :, 3:4]
        ang_velocity = x[:, :, 4:]

        velocity = velocity.permute(0, 2, 1)
        tortuosity = tortuosity.permute(0, 2, 1)
        ang_velocity = ang_velocity.permute(0, 2, 1)

        #Permutes to desired shape
        velocity_encoded = self.velocity_branch(velocity)
        tortuosity_encoded = self.tortuosity_branch(tortuosity)
        ang_velocity_encoded = self.ang_velocity_branch(ang_velocity)

        #Concatenates encoded features
        combined_encoded = torch.cat((velocity_encoded, tortuosity_encoded, ang_velocity_encoded), dim = 1)

        return combined_encoded



In [5]:
#Initialises the model
model = BranchingAE()

#Loads the pretrained weights
model.load_state_dict(torch.load("/Users/maks/Documents/MSc_project/data/models/branching_ae_long_test.pth"))

#Sets the model to evaluation mode
model.eval()


BranchingAE(
  (velocity_branch): Sequential(
    (0): Conv1d(3, 32, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ELU(alpha=1.0)
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv1d(32, 64, kernel_size=(3,), stride=(1,), padding=(1,))
    (4): ELU(alpha=1.0)
    (5): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (tortuosity_branch): Sequential(
    (0): Conv1d(1, 32, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ELU(alpha=1.0)
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv1d(32, 64, kernel_size=(3,), stride=(1,), padding=(1,))
    (4): ELU(alpha=1.0)
    (5): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (ang_velocity_branch): Sequential(
    (0): Conv1d(3, 32, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ELU(alpha=1.0)
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode

In [7]:
no_samples = 1024
batch_size = 32

dataset = TimeSeriesData(no_samples)
dataloader = DataLoader(dataset, batch_size = batch_size, shuffle = True)

The file used is: white95.xlsx


In [19]:
total_loss = 0.0
num_batches = 0
criterion = nn.SmoothL1Loss()

total_loss = 0.0
num_batches = 0
all_outputs = []
all_targets = []

with torch.no_grad():  #No need to compute gradients for evaluation
    for batch_data in dataloader:
        
        #Forward pass
        outputs = model(batch_data)
        
        #Calculates Smooth L1 Loss
        loss = criterion(outputs, batch_data)
        total_loss += loss.item()
        num_batches += 1

        #Collects outputs and targets for correlation calculation
        all_outputs.append(outputs.numpy())
        all_targets.append(batch_data.numpy())

#Concatenates the list into a single NumPy array
all_outputs = np.concatenate(all_outputs, axis=0)
all_targets = np.concatenate(all_targets, axis=0)

#Flattens the arrays if needed 
all_outputs_flat = all_outputs.flatten()
all_targets_flat = all_targets.flatten()

#Calculates average loss
average_loss = total_loss / num_batches
print(f"Average Reconstruction Error (Smooth L1 Loss) on Test Set: {average_loss:.4f}")

#Calculates Pearson correlation coefficient
pearson_corr, _ = pearsonr(all_outputs_flat, all_targets_flat)
print(f"Pearson Correlation Coefficient: {pearson_corr:.4f}")

#Calculates Spearman correlation coefficient
spearman_corr, _ = spearmanr(all_outputs_flat, all_targets_flat)
print(f"Spearman Correlation Coefficient: {spearman_corr:.4f}")

Average Reconstruction Error (Smooth L1 Loss) on Test Set: 0.0042
Pearson Correlation Coefficient: 0.9999
Spearman Correlation Coefficient: 0.9872


In [31]:
#Evaluation loop parameters
no_samples = 1024
batch_size = 32
no_loops = 5

#Initialises storage for results
loss_vals = []
pearson_vals = []
spearman_vals = []

#Initialises loss function
criterion = nn.SmoothL1Loss()

for _ in range(no_loops):

    #Generates dataset
    dataset = TimeSeriesData(no_samples)
    dataloader = DataLoader(dataset, batch_size = batch_size, shuffle = True)

    total_loss = 0.0
    no_batches = 0
    all_outputs = []
    all_targets = []

    with torch.no_grad(): #Gradients unnecessary for evaluation
        for batch_data in dataloader:
            
            #Gets reconstructed outputs
            outputs = model(batch_data)

            #Calculates Smooth L1 loss
            loss = criterion(outputs, batch_data)
            total_loss += loss.item()
            no_batches += 1

            #Collects outputs and targets to calculate correlations
            all_outputs.append(outputs.numpy())
            all_targets.append(batch_data.numpy())

    #Concatenates list into a single array
    all_outputs = np.concatenate(all_outputs, axis = 0)
    all_targets = np.concatenate(all_targets, axis = 0)

    #Flattens arrays
    all_outputs_flat = all_outputs.flatten()
    all_targets_flat = all_targets.flatten()

    #Calculates average loss 
    average_loss = total_loss / no_batches
    loss_vals.append(average_loss)

    #Calculates Pearson correlation coefficient
    pearson, _ = pearsonr(all_outputs_flat, all_targets_flat)
    pearson_vals.append(pearson)

    #Calculates Spearman correlation coefficient
    spearman, _ = spearmanr(all_outputs_flat, all_targets_flat)
    spearman_vals.append(spearman)

#Calculates average and standard deviation for all iterations
avg_loss = np.mean(loss_vals)
std_loss = np.std(loss_vals)

avg_pearson = np.mean(pearson_vals)
std_pearson = np.std(pearson_vals)

avg_spearman = np.mean(spearman_vals)
std_spearman = np.std(spearman_vals)

#Prints results
print(f"Over {no_loops} runs:")
print(f"Average reconstruction error (Smooth L1 Loss): {avg_loss:.4f} with a standard deviation of {std_loss:.6f}.")
print(f"Average Pearson correlation coefficient: {avg_pearson:.4f} with a standard deviation of {std_pearson:.6f}.")
print(f"Average Spearman correlation coefficient: {avg_spearman:.4f} with a standard deviation of {std_spearman:.6f}.")


The file used is: untagged9.xlsx
The file used is: green68.xlsx
The file used is: red61.xlsx
The file used is: untagged4.xlsx
The file used is: yellow44.xlsx
Over 5 runs:
Average reconstruction error (Smooth L1 Loss): 36947788423.4247 with a standard deviation of 64549017548.398773.
Average Pearson correlation coefficient: 0.9896 with a standard deviation of 0.014286.
Average Spearman correlation coefficient: 0.9484 with a standard deviation of 0.053305.


In [30]:
#Prints results
print(f"Over {no_loops} runs:")
print(f"Average reconstruction error (Smooth L1 Loss): {avg_loss:.6f} with a standard deviation of {std_loss:.6f}.")
print(f"Average Pearson correlation coefficient: {avg_pearson:.6f} with a standard deviation of {std_pearson:.6f}.")
print(f"Average Spearman correlation coefficient: {avg_spearman:.6f} with a standard deviation of {std_spearman:.6f}.")


Over 5 runs:
Average reconstruction error (Smooth L1 Loss): 0.005494 with a standard deviation of 0.004697.
Average Pearson correlation coefficient: 0.999756 with a standard deviation of 0.000040.
Average Spearman correlation coefficient: 0.987966 with a standard deviation of 0.006661.
